# On-Axis Field of a Solenoid Demo

This demo compares the numerically computed on-axis magnetic field of a model solenoid with the known analytical formula.

In [1]:
import numpy as np
from em_app.currentcoils import RingCoil
import matplotlib.pyplot as plt
from mtflib import mtf

mtf.initialize_mtf(max_order=6, max_dimension=4)

## 1. Visualize the Geometry

In [2]:
solenoid_radius = 0.1
solenoid_length = 0.5
num_rings_vis = 10

loops = []
ring_positions_z_vis = np.linspace(
    -solenoid_length / 2, solenoid_length / 2, num_rings_vis
)
axis_direction = np.array([0, 0, 1])
for z_pos in ring_positions_z_vis:
    center_point = np.array([0, 0, z_pos])
    loops.append(RingCoil(1.0, solenoid_radius, 20, center_point, axis_direction))

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
for loop in loops:
    loop.plot(ax)
ax.set_title("Geometry of the Solenoid Model")
plt.show()

## 2. Numerical B-Field Calculation

In [ ]:
# --- Numerical B-Field Calculation ---
# Model the solenoid as a collection of rings
num_rings_model = 100
current = 1.0

solenoid_loops_model = []
ring_positions_z_model = np.linspace(-solenoid_length / 2, solenoid_length / 2, num_rings_model)
axis_direction = np.array([0, 0, 1])
for z_pos in ring_positions_z_model:
    center_point = np.array([0, 0, z_pos])
    solenoid_loops_model.append(
        RingCoil(current, solenoid_radius, 20, center_point, axis_direction)
    )

# Define the point on the axis to calculate the field
observation_point = np.array([[0, 0, 0]])

# Sum the contribution from each loop in the solenoid model
total_B_numerical = np.zeros(3, dtype=float)
for loop in solenoid_loops_model:
    b_field_object = loop.biot_savart(observation_point, backend='c')
    _, b_field_numerical = b_field_object._get_numerical_data()
    total_B_numerical += b_field_numerical[0]

print(f"Computed Bz from Solenoid Model: {total_B_numerical[2]}")

## 3. Analytical Formula

In [ ]:
mu_0 = 4 * np.pi * 1e-7
turns_per_meter = num_rings_model / solenoid_length
z1 = -solenoid_length / 2
z2 = solenoid_length / 2
z = 0

cos_theta_1 = (z - z1) / np.sqrt((z - z1) ** 2 + solenoid_radius**2)
cos_theta_2 = (z - z2) / np.sqrt((z - z2) ** 2 + solenoid_radius**2)
B_z_analytical = (mu_0 * turns_per_meter * current / 2) * (cos_theta_1 - cos_theta_2)

print(f"Analytical Bz for Solenoid: {B_z_analytical}")